# Problem 1: Binary Words and Operations

Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the Secure Hash Standard (see page 10).

## Functions to Implement:

1. `Parity(x, y, z)`
2. `Ch(x, y, z)`
3. `Maj(x, y, z)`
4. `Sigma0(x)` - written as Σ₀²⁵⁶(x) in the standard
5. `Sigma1(x)` - written as Σ₁²⁵⁶(x) in the standard
6. `sigma0(x)` - written as σ₀²⁵⁶(x) in the standard
7. `sigma1(x)` - written as σ₁²⁵⁶(x) in the standard

## References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Section 4.1.2, Page 10 - SHA-256 Functions

In [4]:
# ----------------------------------------------------------------------------
# PART 1: Parity Function
# ----------------------------------------------------------------------------
import numpy as np
def Parity(x, y, z):
    """

    Parity function from SHA-256 specification.
    
    Returns the bitwise XOR of x, y, and z.
    Formula: x ⊕ y ⊕ z
    
    This function is used in the SHA-1 algorithm but included here
    for completeness of bitwise operations.
     
    Args:
        x, y, z: 32-bit unsigned integers
    Returns:
        np.uint32: Result of x XOR y XOR z
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Perform XOR operation on all three values
    return np.uint32(x ^ y ^ z)  
# ----------------------------------------------------------------------------
# PART 2: Choice (Ch) Function
# ----------------------------------------------------------------------------


def Ch(x, y, z):
    """
    Choice function from SHA-256 specification.
    
    For each bit position, chooses bit from y if x is 1,
    otherwise chooses bit from z.
    
    Formula: (x ∧ y) ⊕ (¬x ∧ z)
    
    Used in: SHA-256 compression function (Step 1 of each round)
    
    Args:
        x, y, z: 32-bit unsigned integers
    
    Returns:
        np.uint32: Result of the choice operation
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Where x is 1, choose y; where x is 0, choose z
    return np.uint32((x & y) ^ (~x & z))






In [5]:
# Test the Parity function
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)

result = Parity(x, y, z)
print(f"Parity({x}, {y}, {z}) = {result}")
print(f"Binary: {bin(result)}")

Parity(12, 10, 6) = 0
Binary: 0b0


In [6]:
# Test the Ch function
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)

result = Ch(x, y, z)
print(f"Ch({x}, {y}, {z}) = {result}")
print(f"Binary: {bin(result)}")

Ch(12, 10, 6) = 10
Binary: 0b1010
